<a href="https://colab.research.google.com/github/kimbaorrr/object_detection/blob/main/Yolo_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update && apt install ffmpeg libsm6 libxext6 -y
!pip install -U roboflow ultralytics

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
from glob import glob
from roboflow import Roboflow
import os
import random
import math
import numpy as np

In [ ]:
# Thiết đặt các biến môi trường
project_name = 'fire_detection' # @param {type:"string"}
home_path = '/content/drive/MyDrive/Yolo' # @param {type:"string"}
yolo_mode = 'detect' # @param ["detect", "classify", "segment"] {type:"string"}
# Chọn thư mục hoạt động
%cd {home_path}

# Train từ bộ dữ liệu Roboflow


In [ ]:
# Chọn thư mục hoạt động & tạo thư mục chứa dataset của project
!mkdir roboflow

%cd roboflow
!mkdir {project_name}

In [ ]:
# Login & tải data format theo model yolov8
%cd {project_name}

!roboflow login
!roboflow download -f 'yolov8' -l 'data' 'my-workspace-jlq0f/motorbike-license-plate-detection/7'

In [ ]:
# Edit lại file data.yaml

!sed -i 's|train:.*|train: {home_path}/roboflow/{project_name}/data/train/images|' data/data.yaml
!sed -i 's|val:.*|val: {home_path}/roboflow/{project_name}/data/valid/images|' data/data.yaml
!sed -i 's|test:.*|test: {home_path}/roboflow/{project_name}/data/test/images|' data/data.yaml

In [ ]:
# Bắt đầu Train

!yolo {yolo_mode} mode=train data=data/data.yaml epochs=50 imgsz=640 patience=10 optimizer=AdamW batch=-1 exist_ok=True plots=True dropout=0.10

In [ ]:
# Đánh giá quá trình Train

!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
%load_ext tensorboard
%tensorboard --logdir runs/train3

In [ ]:
# Đánh giá mô hình qua tập Valid

!yolo {yolo_mode} mode=val model=runs/detect/train3/weights/best.pt data=data/data.yaml save_json=True

In [ ]:
# Dự đoán thử 10 ảnh từ tập Valid/Test
!yolo {yolo_mode} mode=predict model=runs/detect/train/weights/best.pt conf=0.25 source=data/test/images save=True

for image_path in glob(f'runs/detect/predict/*.jpg')[:10]:
      display(Image(filename=image_path, width=400))
      print('\n')

# Train từ bộ dữ liệu ngoài

In [ ]:
# Chọn thư mục hoạt động & tạo các thư mục cần thiết
!mkdir custom_data

%cd custom_data
!mkdir {project_name}
!mkdir {project_name}/data

In [ ]:
# Giải nén tập dữ liệu
!unzip -jodnq {project_name}/data fire.zip

In [ ]:
# Xóa & khởi tạo lại tệp data.yaml và yolo.names
%cd {project_name}

!rm -f data.yaml

!echo 'train: {home_path}/custom_data/{project_name}/train.txt' >> data.yaml
!echo 'val: {home_path}/custom_data/{project_name}/val.txt' >> data.yaml
!echo 'names: {home_path}/custom_data/{project_name}/yolo.names' >> data.yaml

!echo 'fire' >> yolo.names


In [ ]:
# Tách dữ liệu để Train/Test

!rm -f 'train.txt' 'val.txt'

val_split = .20
files = []

for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob(os.path.join('data', ext))
  files += image_files

nb_val = math.floor(len(files)*val_split)
rand_idx = np.random.randint(0, len(files), nb_val)

# Tạo file train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

# Tạo file val.txt
with open("val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

print('THỐNG KÊ DATASET')
print(f'TRAIN: {len(files) - nb_val}')
print(f'VAL: {nb_val}')


In [ ]:
# Bắt đầu Train

!yolo {yolo_mode} mode=train model=yolov8n.pt data=data.yaml epochs=50 imgsz=640 patience=10 optimizer=AdamW batch=-1 exist_ok=True plots=True dropout=0.10

In [ ]:
# Đánh giá quá trình Train

!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
%load_ext tensorboard
%tensorboard --logdir runs/detect/train